# Export data for ML training

Convert Pixeltable data to PyTorch DataLoader format for model training.


## Problem

You have prepared training data—images with labels, text with embeddings, or multimodal data—and need to export it for PyTorch model training.

| Data type | Use case |
|-----------|----------|
| Images + labels | Image classification |
| Text + embeddings | Fine-tuning embeddings |
| Audio + transcripts | Speech model training |


## Solution

**What's in this recipe:**
- Convert query results to PyTorch Dataset
- Use with DataLoader for batch training
- Export to Parquet for external tools

You use `query.to_pytorch_dataset()` to create an iterable dataset compatible with PyTorch DataLoader.


### Setup


In [ ]:
%pip install -qU pixeltable torch torchvision


In [ ]:
import pixeltable as pxt
import torch
from torch.utils.data import DataLoader


In [ ]:
# Create a fresh directory
pxt.drop_dir('pytorch_demo', force=True)
pxt.create_dir('pytorch_demo')


### Create sample training data


In [ ]:
# Create table with images and labels
training_data = pxt.create_table(
    'pytorch_demo.training_data',
    {'image': pxt.Image, 'label': pxt.Int}
)


In [ ]:
# Insert sample images with labels
base_url = 'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/images'
samples = [
    {'image': f'{base_url}/000000000036.jpg', 'label': 0},  # cat
    {'image': f'{base_url}/000000000090.jpg', 'label': 1},  # other
    {'image': f'{base_url}/000000000139.jpg', 'label': 1},  # other
]
training_data.insert(samples)


### Export to PyTorch Dataset


In [ ]:
# Convert to PyTorch dataset
# 'pt' format returns images as CxHxW tensors with values in [0,1]
pytorch_dataset = training_data.select(
    training_data.image, 
    training_data.label
).to_pytorch_dataset(image_format='pt')

type(pytorch_dataset)  # Should be IterableDataset


In [ ]:
# Use with PyTorch DataLoader
dataloader = DataLoader(pytorch_dataset, batch_size=2)

# Get first batch to verify shape
batch = next(iter(dataloader))
batch['image'].shape, batch['label']  # (batch_size, C, H, W), labels


### Export to Parquet for external tools


In [ ]:
import tempfile
from pathlib import Path

# Export to Parquet for use with other ML tools
export_path = Path(tempfile.mkdtemp()) / 'training_data'

pxt.io.export_parquet(
    training_data.select(training_data.label),  # Non-image columns
    parquet_path=export_path
)


## Explanation

**Export methods:**

| Method | Output | Use case |
|--------|--------|----------|
| `to_pytorch_dataset()` | PyTorch IterableDataset | Direct training |
| `export_parquet()` | Parquet files | External tools, sharing |
| `export_lancedb()` | LanceDB | Vector search apps |
| `to_coco_dataset()` | COCO JSON | Object detection |

**Image format options:**

| Format | Shape | Values | Use |
|--------|-------|--------|-----|
| `'pt'` | CxHxW | [0, 1] float32 | PyTorch models |
| `'np'` | HxWxC | [0, 255] uint8 | NumPy processing |

**DataLoader tips:**
- Data is cached to disk for efficient repeated loading
- Use `num_workers > 0` for parallel data loading
- Filter/transform data before export to reduce size


## See also

- [Sample data for training](./data-sampling.ipynb) - Stratified sampling
- [Import Parquet files](./data-import-parquet.ipynb) - Parquet import/export
